<a href="https://colab.research.google.com/github/hmm0403/TIL/blob/main/221109_trans_1hour_sample37.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
#Import the necessary data science libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Importing our TensorFlow libraries
from math import sqrt
from pandas import DataFrame
from pandas import concat
from matplotlib import pyplot
import sklearn
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from sklearn.preprocessing import StandardScaler
import pickle
import joblib

In [2]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [3]:
#Import the data set as a pandas DataFrame
df = pd.read_csv('입력데이터.csv', encoding='cp949')

In [4]:
#Import the data set as a pandas DataFrame
df1 = pd.read_csv('라벨데이터37816.csv', encoding='cp949')

In [5]:
df_index = df[(df["날짜"].between('2018-02-03','2018-02-10'))].index
df = df.drop(df_index)
df = df.reset_index(drop=True)
df

,날짜,시:분,온도 (℃),습도 (%),CO2 (ppm),일사 (w/㎡)
0,2017-10-20,0:15:00,18.5,90.7,651.0,6.8
1,2017-10-20,1:15:00,18.2,90.4,675.5,6.8
2,2017-10-20,2:15:00,17.6,89.8,685.5,7.3
3,2017-10-20,3:15:00,17.2,90.5,722.5,6.8
4,2017-10-20,4:15:00,18.1,90.8,732.5,7.3
...,...,...,...,...,...,...
2539,2018-02-02,19:15:00,14.8,87.0,482.5,7.1
2540,2018-02-02,20:15:00,15.5,85.3,492.5,7.1
2541,2018-02-02,21:15:00,15.3,84.4,515.5,7.3
2542,2018-02-02,22:15:00,16.0,83.4,542.0,7.3


In [6]:
df2 = pd.DataFrame()
for i in range(4):
  df2 = df2.append(df)
df2 = df2.reset_index(drop=True)
df2

<ipython-input-6-c025cffaa333>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df)
<ipython-input-6-c025cffaa333>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(df)


,날짜,시:분,온도 (℃),습도 (%),CO2 (ppm),일사 (w/㎡)
0,2017-10-20,0:15:00,18.5,90.7,651.0,6.8
1,2017-10-20,1:15:00,18.2,90.4,675.5,6.8
2,2017-10-20,2:15:00,17.6,89.8,685.5,7.3
3,2017-10-20,3:15:00,17.2,90.5,722.5,6.8
4,2017-10-20,4:15:00,18.1,90.8,732.5,7.3
...,...,...,...,...,...,...
10171,2018-02-02,19:15:00,14.8,87.0,482.5,7.1
10172,2018-02-02,20:15:00,15.5,85.3,492.5,7.1
10173,2018-02-02,21:15:00,15.3,84.4,515.5,7.3
10174,2018-02-02,22:15:00,16.0,83.4,542.0,7.3


In [7]:
df3 = df1.values
df3 = np.ravel(df3, order='F')
df3

array([30. , 27. , 20. , 25. , 20. ,  8. , 40. , 12. , 24. , 13. , 19. ,
       18. , 20. , 19. , 28. , 14. , 26. , 27. , 26. , 25. , 11. , 20. ,
       34. , 10. , 20. , 21. , 18. , 16. , 20. , 18. , 27. , 26. , 27. ,
       26. , 20. , 24. , 20. , 43. ,  5. , 11. , 21. , 23. , 22. , 19. ,
       22. , 25. , 19. , 24. , 32. , 26.5, 18.5, 25. , 26. , 23. , 18. ,
       10. , 35. ,  9. , 25. , 19. , 20.5, 25.5, 21. , 21. ])

In [8]:
dataset = df2.drop(["날짜", "시:분"], axis=1)
dataset = dataset.reset_index(drop=True)
dataset

,온도 (℃),습도 (%),CO2 (ppm),일사 (w/㎡)
0,18.5,90.7,651.0,6.8
1,18.2,90.4,675.5,6.8
2,17.6,89.8,685.5,7.3
3,17.2,90.5,722.5,6.8
4,18.1,90.8,732.5,7.3
...,...,...,...,...
10171,14.8,87.0,482.5,7.1
10172,15.5,85.3,492.5,7.1
10173,15.3,84.4,515.5,7.3
10174,16.0,83.4,542.0,7.3


In [9]:
df1 = df1.values
df1

array([[30. , 26. , 27. , 32. ],
       [27. , 27. , 26. , 26.5],
       [20. , 26. , 20. , 18.5],
       [25. , 25. , 24. , 25. ],
       [20. , 11. , 20. , 26. ],
       [ 8. , 20. , 43. , 23. ],
       [40. , 34. ,  5. , 18. ],
       [12. , 10. , 11. , 10. ],
       [24. , 20. , 21. , 35. ],
       [13. , 21. , 23. ,  9. ],
       [19. , 18. , 22. , 25. ],
       [18. , 16. , 19. , 19. ],
       [20. , 20. , 22. , 20.5],
       [19. , 18. , 25. , 25.5],
       [28. , 27. , 19. , 21. ],
       [14. , 26. , 24. , 21. ]])

In [10]:
# dataset
scaler = StandardScaler()
dataset = scaler.fit_transform(dataset)
dataset

array([[ 0.04848716,  0.95551355,  0.45589463, -0.60461754],
       [-0.02490713,  0.92769734,  0.66279392, -0.60461754],
       [-0.17169572,  0.87206491,  0.74724261, -0.60276951],
       ...,
       [-0.7343853 ,  0.37137306, -0.68838513, -0.60276951],
       [-0.56313195,  0.27865235, -0.4645961 , -0.60276951],
       [-0.58759671,  0.22301992, -0.29147628, -0.60276951]])

In [11]:
dataset = dataset.reshape(64,159,4)
print(dataset)

[[[ 0.04848716  0.95551355  0.45589463 -0.60461754]
  [-0.02490713  0.92769734  0.66279392 -0.60461754]
  [-0.17169572  0.87206491  0.74724261 -0.60276951]
  ...
  [ 1.88334449 -1.80756369 -1.49909256  2.99128369]
  [ 2.07906261 -1.95591683 -1.79888541  1.6873121 ]
  [ 1.90780926 -2.54005732 -1.62998803  1.71614141]]

 [[ 1.56530256 -1.95591683 -1.84955462  0.81060558]
  [ 1.41851397 -1.82610783 -2.10712313 -0.12893403]
  [ 0.51331769 -0.08295843 -1.57087394 -0.53143546]
  ...
  [-0.07383666  0.70516763  0.71768557 -0.60276951]
  [-0.09830142  0.74225591  0.87813808 -0.60461754]
  [-0.17169572  0.83497662  1.08081494 -0.60535676]]

 [[-0.22062525  0.88133698  1.26660206 -0.60461754]
  [-0.09830142  0.90915319  1.47350135 -0.51517278]
  [-0.04937189  0.77007213  1.36371805 -0.19657201]
  ...
  [-0.58759671  0.09321092 -1.5539842  -0.60276951]
  [-1.076892    0.13029921 -1.26263622 -0.60535676]
  [-0.44080813  0.69589555 -0.90795172 -0.60535676]]

 ...

 [[-0.90563865  0.31574063  0.5656

In [12]:
df3 = df3.reshape(64,1,1)
print(df3)

[[[30. ]]

 [[27. ]]

 [[20. ]]

 [[25. ]]

 [[20. ]]

 [[ 8. ]]

 [[40. ]]

 [[12. ]]

 [[24. ]]

 [[13. ]]

 [[19. ]]

 [[18. ]]

 [[20. ]]

 [[19. ]]

 [[28. ]]

 [[14. ]]

 [[26. ]]

 [[27. ]]

 [[26. ]]

 [[25. ]]

 [[11. ]]

 [[20. ]]

 [[34. ]]

 [[10. ]]

 [[20. ]]

 [[21. ]]

 [[18. ]]

 [[16. ]]

 [[20. ]]

 [[18. ]]

 [[27. ]]

 [[26. ]]

 [[27. ]]

 [[26. ]]

 [[20. ]]

 [[24. ]]

 [[20. ]]

 [[43. ]]

 [[ 5. ]]

 [[11. ]]

 [[21. ]]

 [[23. ]]

 [[22. ]]

 [[19. ]]

 [[22. ]]

 [[25. ]]

 [[19. ]]

 [[24. ]]

 [[32. ]]

 [[26.5]]

 [[18.5]]

 [[25. ]]

 [[26. ]]

 [[23. ]]

 [[18. ]]

 [[10. ]]

 [[35. ]]

 [[ 9. ]]

 [[25. ]]

 [[19. ]]

 [[20.5]]

 [[25.5]]

 [[21. ]]

 [[21. ]]]


In [13]:
train = dataset
test = df3
print(train)
print(test)

[[[ 0.04848716  0.95551355  0.45589463 -0.60461754]
  [-0.02490713  0.92769734  0.66279392 -0.60461754]
  [-0.17169572  0.87206491  0.74724261 -0.60276951]
  ...
  [ 1.88334449 -1.80756369 -1.49909256  2.99128369]
  [ 2.07906261 -1.95591683 -1.79888541  1.6873121 ]
  [ 1.90780926 -2.54005732 -1.62998803  1.71614141]]

 [[ 1.56530256 -1.95591683 -1.84955462  0.81060558]
  [ 1.41851397 -1.82610783 -2.10712313 -0.12893403]
  [ 0.51331769 -0.08295843 -1.57087394 -0.53143546]
  ...
  [-0.07383666  0.70516763  0.71768557 -0.60276951]
  [-0.09830142  0.74225591  0.87813808 -0.60461754]
  [-0.17169572  0.83497662  1.08081494 -0.60535676]]

 [[-0.22062525  0.88133698  1.26660206 -0.60461754]
  [-0.09830142  0.90915319  1.47350135 -0.51517278]
  [-0.04937189  0.77007213  1.36371805 -0.19657201]
  ...
  [-0.58759671  0.09321092 -1.5539842  -0.60276951]
  [-1.076892    0.13029921 -1.26263622 -0.60535676]
  [-0.44080813  0.69589555 -0.90795172 -0.60535676]]

 ...

 [[-0.90563865  0.31574063  0.5656

In [14]:
# split into input and outputs
train_X, test_X = train[:50], train[50:]
train_y, test_y = test[:50], test[50:]

print(train_X, train_y)

[[[ 0.04848716  0.95551355  0.45589463 -0.60461754]
  [-0.02490713  0.92769734  0.66279392 -0.60461754]
  [-0.17169572  0.87206491  0.74724261 -0.60276951]
  ...
  [ 1.88334449 -1.80756369 -1.49909256  2.99128369]
  [ 2.07906261 -1.95591683 -1.79888541  1.6873121 ]
  [ 1.90780926 -2.54005732 -1.62998803  1.71614141]]

 [[ 1.56530256 -1.95591683 -1.84955462  0.81060558]
  [ 1.41851397 -1.82610783 -2.10712313 -0.12893403]
  [ 0.51331769 -0.08295843 -1.57087394 -0.53143546]
  ...
  [-0.07383666  0.70516763  0.71768557 -0.60276951]
  [-0.09830142  0.74225591  0.87813808 -0.60461754]
  [-0.17169572  0.83497662  1.08081494 -0.60535676]]

 [[-0.22062525  0.88133698  1.26660206 -0.60461754]
  [-0.09830142  0.90915319  1.47350135 -0.51517278]
  [-0.04937189  0.77007213  1.36371805 -0.19657201]
  ...
  [-0.58759671  0.09321092 -1.5539842  -0.60276951]
  [-1.076892    0.13029921 -1.26263622 -0.60535676]
  [-0.44080813  0.69589555 -0.90795172 -0.60535676]]

 ...

 [[-0.90563865  0.54754241  0.1898

In [15]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], train_X.shape[1],4))
test_X = test_X.reshape((test_X.shape[0], test_X.shape[1], 4))
train_y = train_y.reshape((train_y.shape[0], train_y.shape[1],1))
test_y = test_y.reshape((test_y.shape[0], test_y.shape[1],1))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(50, 159, 4) (50, 1, 1) (14, 159, 4) (14, 1, 1)


In [16]:
train_X
train_y
test_X
test_y

array([[[18.5]],

       [[25. ]],

       [[26. ]],

       [[23. ]],

       [[18. ]],

       [[10. ]],

       [[35. ]],

       [[ 9. ]],

       [[25. ]],

       [[19. ]],

       [[20.5]],

       [[25.5]],

       [[21. ]],

       [[21. ]]])

In [17]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads
        # , dropout=dropout
    )(x, x)
    # x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    # x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    # x = layers.Dropout(dropout)(x)
    # x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [18]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    # x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    # for dim in mlp_units:
    #     x = layers.Dense(dim, activation="relu")(x)
    #     x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="relu")(x)
    return keras.Model(inputs, outputs)

In [19]:
input_shape = train_X.shape[1:]
print(input_shape)
model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["mse"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    train_X,
    train_y,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

(159, 4)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 159, 4)]     0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 159, 4)      8           ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 multi_head_attention (MultiHea  (None, 159, 4)      19460       ['layer_normalization[0][0]',    
 dAttention)                                                      'layer_normalization[0][0]']    
                                                                                     

In [20]:
loss_and_metrics = model.evaluate(test_X, test_y, batch_size=32)
print('## evaluation loss and_metrics ##')
print(loss_and_metrics)

x_pred = test_X[:1]
y_pred = model.predict(x_pred)


1/1 [==============================] - 0s 35ms/step - loss: 44.8043 - mse: 44.8043
## evaluation loss and_metrics ##
[44.804325103759766, 44.804325103759766]
1/1 [==============================] - 0s 341ms/step


In [39]:
filename = 'transformer_model.sav'
pickle.dump(model, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.predict(x_pred)
mse = np.mean((result - test_y) ** 2)
print(result)
print("Mean Squared Error (MSE):", mse)

1/1 [==============================] - 0s 302ms/step
[[[18.408949]
  [18.293818]
  [18.455946]
  [19.754808]
  [21.472006]
  [21.956722]
  [19.96176 ]
  [21.076118]
  [20.91165 ]
  [21.168537]
  [20.927942]
  [21.632166]
  [21.120867]
  [20.19845 ]
  [19.660269]
  [19.672169]
  [19.528414]
  [19.371357]
  [19.498705]
  [19.434908]
  [19.331442]
  [19.088198]
  [18.840895]
  [18.553255]
  [18.477371]
  [18.577633]
  [18.772036]
  [20.935188]
  [20.171202]
  [21.333296]
  [21.389807]
  [21.174744]
  [20.279531]
  [20.632433]
  [21.447344]
  [21.480402]
  [20.975632]
  [19.519068]
  [19.502253]
  [19.595129]
  [19.511503]
  [19.167276]
  [19.407598]
  [19.667688]
  [19.57688 ]
  [19.184835]
  [19.104082]
  [19.226084]
  [19.058336]
  [19.107512]
  [19.360506]
  [20.588228]
  [21.866125]
  [22.361906]
  [20.987827]
  [20.945427]
  [20.328146]
  [21.135508]
  [21.413727]
  [22.039164]
  [21.203299]
  [19.803797]
  [19.64079 ]
  [20.260454]
  [19.933435]
  [20.09318 ]
  [20.274965]
  [20.261